## NEP_Elastic_properties

In [1]:
from ase.calculators.mixing import SumCalculator
from ase.io import read
from calorine.calculators import CPUNEP
from calorine.tools import get_elastic_stiffness_tensor, relax_structure
from dftd3.ase import DFTD3

import sys
import os
import numpy as np

def nep_calculator(filename="./nep.txt", use_dftd3=False):
    """
    配置NEP计算器，若需色散校正，则use_dftd3=True
    """
    if os.path.exists(filename):
        if use_dftd3:
            return SumCalculator([CPUNEP(filename), DFTD3(method="pbe", damping="d3bj")])
        return CPUNEP(filename)
    print(f"File {filename} does not exist.")
    sys.exit(1)

def calculate_moduli(cij):
    """
    根据弹性常数矩阵cij计算体积模量、剪切模量和杨氏模量。
    """
    c11, c12, c44 = cij[0, 0], cij[0, 1], cij[3, 3]
    
    bulk_modulus = (c11 + 2 * c12) / 3
    shear_modulus = (c11 - c12 + 3 * c44) / 5
    youngs_modulus = (9 * bulk_modulus * shear_modulus) / (3 * bulk_modulus + shear_modulus)

    return bulk_modulus, shear_modulus, youngs_modulus

def check_stability_rhombohedral(cij):
    """
    计算菱方相的机械稳定性条件值K1, K2, K3。
    """
    c11, c12, c13, c33, c44, c14 = (
        cij[0, 0], cij[0, 1], cij[0, 2],
        cij[2, 2], cij[3, 3], cij[0, 3]
    )
    
    K1 = c11 - c12
    K2 = 0.5 * (c11 + c12) * c33 - c13**2
    K3 = 0.5 * (c11 - c12) * c44 - c14**2

    return K1, K2, K3

In [2]:
structure = read("./POSCAR")
structure.calc = nep_calculator(filename="../nep.txt")

relax_structure(structure, fmax=0.0001)

cij = get_elastic_stiffness_tensor(structure, clamped=False, epsilon=0.01)
with np.printoptions(precision=1, suppress=True):
    print("Elastic stiffness tensor (GPa):")
    print(cij)

bulk_modulus, shear_modulus, youngs_modulus = calculate_moduli(cij)
print(f"\nBulk Modulus (GPa): {bulk_modulus:.1f}")
print(f"Shear Modulus (GPa): {shear_modulus:.1f}")
print(f"Young's Modulus (GPa): {youngs_modulus:.1f}")

K1, K2, K3 = check_stability_rhombohedral(cij)
print(f"\nStability Conditions for Rhombohedral Phase:")
print(f"K1 (GPa): {K1:.1f}")
print(f"K2 (GPa): {K2:.1f}")
print(f"K3 (GPa): {K3:.1f}")

is_stable = all([K1 > 0, K2 > 0, K3 > 0])
print(f"\nIs the rhombohedral structure mechanically stable? {'Yes' if is_stable else 'No'}")

Elastic stiffness tensor (GPa):
[[ 76.9  22.7  25.2 -11.4  -0.   -0. ]
 [ 22.7  76.9  25.2  11.4  -0.   -0. ]
 [ 25.2  25.2  45.3  -0.   -0.   -0. ]
 [-11.4  11.4  -0.   20.5  -0.    0. ]
 [ -0.   -0.   -0.   -0.   20.5 -11.4]
 [ -0.   -0.   -0.    0.  -11.4  27.1]]

Bulk Modulus (GPa): 40.8
Shear Modulus (GPa): 23.2
Young's Modulus (GPa): 58.4

Stability Conditions for Rhombohedral Phase:
K1 (GPa): 54.2
K2 (GPa): 1618.6
K3 (GPa): 427.1

Is the rhombohedral structure mechanically stable? Yes
